In [6]:
from sqlalchemy import create_engine, inspect
import mlflow
import pandas as pd


### Parametros conexiones

In [7]:
db_config = {
    "host": "10.43.101.189",
    "port": 3306,
    "user": "taller-mlflow",
    "password": "mlflow",
    "database": "taller",
}

tracking_uri = "http://10.43.101.189:5000"

### Carga de datos desde servidor de `MySql`

In [8]:
engine = create_engine(
    f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
)

# Crear el inspector de la base de datos
inspector = inspect(engine)

# Obtener la lista de tablas en la base de datos 'taller'
tables = inspector.get_table_names()

# Imprimir la lista de tablas
print("Tablas en la base de datos 'taller':", tables)


Tablas en la base de datos 'taller': ['iris_cleaned', 'iris_raw']


In [10]:
df = pd.read_sql_table('iris_cleaned', engine)

df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


### Set up MLflow

In [11]:
# Set up MLflow tracking URI
tracking_uri = "http://10.43.101.189:5000"
mlflow.set_tracking_uri(tracking_uri)

# Create an experiment
experiment_name = "iris_classification_experiment"
mlflow.set_experiment(experiment_name)

2025/03/16 21:12:16 INFO mlflow.tracking.fluent: Experiment with name 'iris_classification_experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflows3/artifacts/1', creation_time=1742159536746, experiment_id='1', last_update_time=1742159536746, lifecycle_stage='active', name='iris_classification_experiment', tags={}>